# 图像上的算术运算
图像的几种算术运算，例如加法，减法，按位运算

In [1]:
import numpy as np
import cv2 as cv

## 图像的加法
可以通过OpenCV函数``cv.add()``或仅通过numpy操作``res = img1 + img2``添加两个图像。两个图像应具有相同的深度和类型，或者第二个图像可以只是一个标量值。

**注意** OpenCV加法和Numpy加法之间有区别。OpenCV加法是饱和运算，而Numpy加法是模运算。

In [2]:
x = np.uint8([250])
y = np.uint8([10])
print(cv.add(x, y)) # 250 + 10 = 260 => 255
print(x + y)        # 250 + 10 % 256 = 4 

[[255]]
[4]


## 图像融合
同样是图像加法，但是对图像赋予不同的权重，以使其具有融合或透明的感觉。根据以下等式添加图像：
$$G(x)=( 1-\alpha )f_0(x) + \alpha f_1(x)$$
通过改变 $\alpha$ 从0 -> 1， 可以让图像很酷过渡

比如将两幅图像融合在一起。第一幅图像的权重为0.7，第二幅图像的权重为0.3。`cv.addWeighted()`在图像上应用以下公式
$$dst=\alpha \cdot img1 + \beta \cdot img2 + \gamma$$ 
在这里$\gamma$被视作0

In [6]:
img1 = cv.imread('./images/messi1.png')
img1 = img1[26:560, 284:915]
img2 = cv.imread('./images/logo.png')
dst = cv.addWeighted(img1, 0.7, img2, 0.3, 0)
cv.imshow('image', dst)
cv.waitKey(0)
cv.destroyAllWindows()

[![5cvZQg.png](https://z3.ax1x.com/2021/10/22/5cvZQg.png)](https://imgtu.com/i/5cvZQg)

## 按位运算
这包括按位 AND、 OR、NOT 和 XOR 操作。它们在提取图像的任何部分(我们将在后面的章节中看到)、定义和处理非矩形 ROI 等方面非常有用。 下面我们将看到一个例子，如何改变一个图像的特定区域。 

我想把 OpenCV 的标志放在一个图像上面。如果我添加两个图像，它会改变颜色。如果我混合它，我得到一个透明的效果。但我希望它是不透明的。如果是一个矩形区域，我可以使用 ROI，就像我们在上一章中所做的那样。但是 OpenCV 的 logo 不是长方形的。所以你可以使用如下的按位操作来实现:

`cv.threshold()`<br>
[![5gCvN9.png](https://z3.ax1x.com/2021/10/23/5gCvN9.png)](https://imgtu.com/i/5gCvN9) [![5gP9c6.png](https://z3.ax1x.com/2021/10/23/5gP9c6.png)](https://imgtu.com/i/5gP9c6)

In [22]:
img1 = cv.imread('./images/messi1.png')
img2 = cv.imread('./images/logo2.png')

# 图像放在左上角 创建ROI
rows, cols, channels = img2.shape
roi = img1[0:rows, 0:cols]

#创建logo的mask 和 相反掩码
img2gray = cv.cvtColor(img2, cv.COLOR_BGR2GRAY)
cv.imshow('image',img2gray)
cv.waitKey(0)
ret, mask = cv.threshold(img2gray, 254, 255, cv.THRESH_BINARY_INV) 
# 图像二值化 将图像上的像素点的灰度值设置为0或255，这样将使整个图像呈现出明显的黑白效果。
# 在数字图像处理中，二值图像占有非常重要的地位，图像的二值化使图像中数据量大为减少，从而能凸显出目标的轮廓
cv.imshow('image', mask)
cv.waitKey(0)
mask_inv = cv.bitwise_not(mask)
cv.imshow('image', mask_inv)
cv.waitKey(0)
cv.destroyAllWindows()

[![5gkVN4.png](https://z3.ax1x.com/2021/10/23/5gkVN4.png)](https://imgtu.com/i/5gkVN4)
[![5gkZ4J.png](https://z3.ax1x.com/2021/10/23/5gkZ4J.png)](https://imgtu.com/i/5gkZ4J)
[![5gkmC9.png](https://z3.ax1x.com/2021/10/23/5gkmC9.png)](https://imgtu.com/i/5gkmC9)

In [24]:
# bitwise_and发生的是，蒙版的像素值为零(黑色)的空间位置在结果图像中变为像素值零。蒙版的像素值不为0的位置，结果图像将保留其原始灰度值。
# 将ROI中的logo区域涂黑(0)
img1_bg = cv.bitwise_and(roi, roi, mask=mask_inv)
# 将logo颜色抠出
img2_fg = cv.bitwise_and(img2, img2, mask=mask)
dst = cv.add(img1_bg, img2_fg)
img1[0:rows, 0:cols] = dst
cv.imshow('image', img1)
cv.waitKey(0)
cv.destroyAllWindows()

[![5gkBb8.png](https://z3.ax1x.com/2021/10/23/5gkBb8.png)](https://imgtu.com/i/5gkBb8)